Imports

In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os


import load_data as ld
import network_model as nt
import utilities as tools

Load data

In [2]:
# declare consts of main paths
DATABASE = os.path.abspath('../../Database/')
DATABASES_VERSIONS = os.listdir(DATABASE)
# choose the path to right database version
DATABASE_VERSION = 'V1'
use_database = os.path.join(DATABASE, list(filter(lambda x: DATABASE_VERSION in x, DATABASES_VERSIONS))[0])
datasets = ld.get_classes_names(use_database)

If datasets are already divided in to train and validation => run below, if not the next one or no one bc may be not written yet

In [3]:
path_train = os.path.join(use_database, datasets[0])
path_valid = os.path.join(use_database, datasets[1])

classes = ld.get_classes_names(path_train)

# classes numbers
classes_numbers = tools.make_labels_dict(classes)

# train set
imgs_train = []
labels_train = []
# validation set
imgs_valid = []
labels_valid = []
# indicies for labels - kind of iterators
label_train_start = 0
label_valid_start = 0
for class_name in classes:
    # train images
    temp_path = os.path.join(path_train, class_name)
    temp = ld.load_images(temp_path)
    imgs_train.extend(temp)
    for i in range(label_train_start, label_train_start+len(temp)):
        labels_train.append(classes_numbers[class_name])
    label_train_start+=len(temp)
    
    # validation images
    temp_path = os.path.join(path_valid, class_name)
    temp = ld.load_images(temp_path)
    imgs_valid.extend(temp)
    for i in range(label_valid_start, label_valid_start+len(temp)):
        labels_valid.append(classes_numbers[class_name])
    label_valid_start+=len(temp)

# shuffle data
imgs_train, labels_train = tools.shuffle_data(imgs_train, labels_train)
imgs_valid, labels_valid = tools.shuffle_data(imgs_valid, labels_valid)

# convert to numpy array
imgs_train = np.asarray(imgs_train)
labels_train = np.asarray(labels_train)
imgs_valid = np.asarray(imgs_valid)
labels_valid = np.asarray(labels_valid)

Learn model

In [4]:
# model training
# Load the training data

# # Fit the model
img_shape = imgs_train.shape
model = nt.build_neural_model(img_shape = (img_shape[1], img_shape[2], 1), nuber_of_classes = len(classes))
trained_model, history_model = nt.train_model(model, imgs_train=imgs_train, labels_train=labels_train, imgs_val=imgs_valid, labels_val=labels_valid, epochs_number=150)

(48, 48, 1)
Epoch 1/150


c:\Users\peter\anaconda3\envs\inz\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


900/900 [==============================] - 60s 66ms/step - loss: 1.7869 - accuracy: 0.2629 - val_loss: 1.6031 - val_accuracy: 0.3721
Epoch 2/150
900/900 [==============================] - 60s 66ms/step - loss: 1.5874 - accuracy: 0.3757 - val_loss: 1.4602 - val_accuracy: 0.4389
Epoch 3/150
900/900 [==============================] - 60s 66ms/step - loss: 1.4694 - accuracy: 0.4284 - val_loss: 1.3349 - val_accuracy: 0.4858
Epoch 4/150
900/900 [==============================] - 60s 67ms/step - loss: 1.3903 - accuracy: 0.4625 - val_loss: 1.3167 - val_accuracy: 0.4996
Epoch 5/150
900/900 [==============================] - 60s 67ms/step - loss: 1.3222 - accuracy: 0.4892 - val_loss: 1.2576 - val_accuracy: 0.5203
Epoch 6/150
900/900 [==============================] - 60s 67ms/step - loss: 1.2805 - accuracy: 0.5104 - val_loss: 1.1960 - val_accuracy: 0.5425
Epoch 7/150
900/900 [==============================] - 60s 67ms/step - loss: 1.2415 - accuracy: 0.5220 - val_loss: 1.1770 - val_accuracy: 0.54

KeyboardInterrupt: 

Show trained model and evaluate

Serialize model to json

In [ ]:
tools.serialize_model(trained_model, file_name = "yt_guy")

In [ ]:
plt.plot(history_model.history['accuracy'])
plt.plot(history_model.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Make predictions
# predictions = model.predict(X_test)

# eval = model.evaluate(imgs_test, labels_test)

Deserialize model from json

In [ ]:
# cam_access = cv2.VideoCapture(1)

# ret, frame = cam_access.read()
# plt.imshow(frame)

# cam_access.release()

# print((imgs_train.shape))